# Tutorial 10

# Scipy and a bit of pandas 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize, integrate, interpolate
from scipy.misc import derivative
import pandas as pd

## Data fitting

Let's assume we have a set of measured data as shown below.

In [ ]:
x = np.asarray([0, 0.444, 0.889, 1.33, 1.778, 2.222, 2.667, 3.111, 3.556, 4.])
y = np.asarray([0, 0.546, 1.29, 1.85, 2.24, 2.689, 2.916,  2.999, 2.935, 2.728])
y_err = np.asarray([0.26, 0.41, 0.24, 0.49, 0.21, 0.45, 0.66, 0.67 , 0.26, 0.27])

plt.errorbar(x,y, yerr=y_err, marker='o', linestyle='none')

1. What do we need to know in order to fit this data?

2. How to add errors to the fit?
Before running the code, what information can we use to help optimizer to find a solution faster? Check the plot above.

In [ ]:
def f(x, a, b, c):
    return a*(x - b)**2 + c

parameters, covariance = optimize.curve_fit(f, x, y)
fitted_a, fitted_b, fitted_c = parameters

3. How to add the curve with best-fit parameters to the plot?

In [ ]:


plt.errorbar(x,y, yerr=y_err, marker='o', linestyle='none', label='data')
plt.plot(, label='cosine fit', color='r', lw=2,alpha=0.7)

print(parameters)

## Minimization

In the previous example we assumed that we know the fitting function. In many real examples, we don't! For this we usually test different functions and decide which one fits better.

4. Do you any way how to judge which one of two fitted functions fit the data better?

Likely, the most common way to use minimization in physics is for finding best fits, for example, minimize distance between the data point and theoretical model. In the methods like maximum likelihood estimation, we actually *minimize* the negative likelihood.

Let's now test two functions to fit the data, one is a parabola and another is a sine.

In [ ]:
def f1(x, a, b, c):
    return a*(x - b)**2 + c

def f2(x, a, b, c):
    return a*np.sin(b*x) + c

As a criterion of which function fits better we will use

$$\chi^2_r = \frac{1}{N_{data} - N_{par} + 1} \sum \frac{(y_{data} - y_{model})^2}{\sigma^2}$$

`scipy.optimize.minimize` takes a function with a **single** argument, that is why parameters must be defined as structured data.

5. How to change a functions below to take list/tuple `params` instead of `a`,`b`,`c`?

In [ ]:

def chi2_f1(x,a,b,c):
    y_model = f1(x,a,b,c)
    dof = len(x) - 3 + 1
    return np.sum((y - y_model)**2/y_err**2)/dof

def chi2_f2(x,a,b,c):
    y_model = f2(x,a,b,c)
    dof = len(x) - 3 + 1
    return np.sum((y - y_model)**2/y_err**2)/dof

6. Can you suggest an initial guess for each $\chi^2$ function?

In [ ]:
initial1 = []

result1 = optimize.minimize(chi2_f1, initial1)
print(result1.x)
print(result1.fun)

In [ ]:
initial2 = []
result2 = optimize.minimize(chi2_f2, initial2)
print(result2.x)
print(result2.fun)

7. Based on the previously found results, which function fits better?

In [ ]:
a1,b1,c1 = result1.x
a2,b2,c2 = result2.x


f1_v = f1(x, a1, b1, c1)
f2_v  = f2(x, a2, b2, c2)

plt.errorbar(x,y, yerr=y_err, marker='o', linestyle='none', label='data', color='k')
plt.plot(x, f1_v, '-', color='r', lw=2,alpha=0.7, label='parabola')
plt.plot(x, f2_v, '-', color='b', lw=2,alpha=0.7, label='sine')
plt.legend()

## Interpolation

8. What is interpolation? In what cases do we need it?

9. What is extrapolation? When do we need it?

Let's imagine that we numerically calculated some model values. Numerically means that we always operate with a finite amount of points. Now we compare our model to the data and want to estimate how good is the fit, e.g. by calculating $\chi^2$. See the plot below.

In [ ]:
calc_x = np.linspace(0, 5, 15)
calc_y = f2(calc_x,a_b, b_b, c_b)

In [ ]:
plt.errorbar(x,y, yerr=y_err, marker='o', linestyle='none', label='data')
plt.scatter(calc_x, calc_y, color='r')

10. What is the algorithm to calculate $\chi^2$ given that you have two sets of points: ($x_{data}$ , $y_{data}$) and ($x_{model}$ , $y_{model}$)?

11. How to fix the following code?

In [ ]:
spl = interpolate.CubicSpline(x, y)

12. We want to check that interoplation works as expected by plotting the points between the original theoretical model points. What to change in the following code to do this?

In [ ]:
spline_x = np.linspace(0, 5, 100)
plt.errorbar(x,y, yerr=y_err, marker='o', linestyle='none', label='data')
plt.plot(spline_x, spl(x), color='r')

13. How to calculate the value of $\chi^2$ using the previously defined spline function?

In [ ]:
spl = interpolate.CubicSpline(calc_x, calc_y)

(1/(len(x)-3+1))*np.sum()

## Mechanical motion

Imagine that a position of the body is defined as a function of time as

$$ x (t) = t^3 - 4t^2 + 5t -2$$

14. How to find its velosity and acceleration as a function of time?

15. What is wrong in the following code snippet and how to fix it?

In [ ]:
def x(t):
    return t**3 - 10*t**2 - 10*t + 2

def v(t):
    return derivative(x, t, dx=1e-3)

def a(t):
    return derivative(x, t, dx=1e-3)

In [ ]:
t = np.linspace(0, 10, 100)
plt.plot(t,x(t), label='x, m')
plt.plot(t,v(t), label='v, m/c')
plt.plot(t,a(t), label=r"a, m/s$^2$")
plt.xlabel('t, s')
plt.ylabel('displacement, speed, acceleration')
plt.grid()
plt.legend()

## Center of the mass calculation

Imagine there is a cone with radius R = 1 and height H = 4. It is made from non-homogeneous material which density depends only on the height as

$$\rho (z) = \frac{5}{z+1}$$

The center of mass can be calculated as

$$z_{CM} = \frac{1}{M} \int_0^H z \rho dV = \frac{1}{M} \int_0^H z \rho(z) A(z) dz $$

M is the total mass; $A(z)$ can be expressed as

$$A(z) = \pi r(z)^2$$
with
$$r(z) = R (1 - \frac{z}{H})$$

16. How to calculate total mass?

17. The expression under the integral is a multipliation of 2-3 functions. How to integrate it **without** defining new function?

In [ ]:
R = 1
H = 4

def rho(z):
    return 5/(z+1)

def area(z):
    radius = R * (1 - z / h)
    return np.pi * radius**2



M, M_err = integrate.quad(, 0, H)
z, z_err = integrate.quad(, 0, H)


In [ ]:
print(f"Total mass: {M}, with precision up to {M_err}")
print(f"Center of the mass is located at {z/M:.2f} cm, with precision up to {z_err/M:.2f} cm")

# Befriend pandas

One of the most useful libraries to work with data is `pandas`. We will friefly look into:
- saving data into csv
- loading csv
- basic operations with series

You will learn more during the next lecture!

18. Imagine you have a data that you want to send to your friend. What format would you choose and why?

In [ ]:
x = np.asarray([0, 0.444, 0.889, 1.33, 1.778, 2.222, 2.667, 3.111, 3.556, 4.])
y = np.asarray([0, 0.546, 1.29, 1.85, 2.24, 2.689, 2.916,  2.999, 2.935, 2.728])
y_err = np.asarray([0.26, 0.41, 0.24, 0.49, 0.21, 0.45, 0.66, 0.67 , 0.26, 0.27])

The easiest way to convert a couple of arrays into `pandas.DataFrame` is either via dictionary or by `data = [x, y, y_err]` and manually add column names to the DataFrame.  

In [ ]:
data = {'x':x, 'y':y, 'y_err':y_err}
df = pd.DataFrame(data)

In [ ]:
df

We can easili save as as `.csv` by

In [ ]:
df.to_csv('ex1_data.csv', header=list(df.columns.values), index=False)

`pandas` is the most compact and convenient way to work with table-like structures (recall all your file reading/writing experience, data type conversions and `csv.reader`). We load CSV as following

In [ ]:
df2 = pd.read_csv('ex1_data.csv')

In [ ]:
df2

Operations with the columns look similar to operations with dictionaries. Each column (called series) is a numpy array.

In [ ]:
df2['x']

In [ ]:
df2['x']**2

This is the final tutorial of our 'Python for Physicists' course. Next week marks the conclusion of the lectures. And here is where your journey **begins**!

> If you find that you're spending almost all your time on theory, start turning some attention to practical things; it will improve your theories. If you find that you’re spending almost all your time on practice, start turning some attention to theoretical things; it will improve your practice. 
-- Donald Knuth


Happy programming!